<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/pablo_camarillo_add_spark_setup/examples/notebooks/lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
  .master("local[*]") \
  .appName("ITESO-2024-SparkIntroduction") \
  .config("spark.driver.bindAddress","localhost") \
  .config("spark.ui.port","4040") \
  .getOrCreate()

sc = spark.sparkContext

In [ ]:
# Create an RDD with a list of sentences
sentences = [
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit.",
    "Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.",
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur.",
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
    "Curabitur pretium tincidunt lacus. Nulla gravida orci a odio.",
    "Nullam varius, turpis et commodo pharetra, est eros bibendum elit, nec malesuada elit elit vel lectus.",
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium.",
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit.",
    "Sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt.",
    "Neque porro quisquam est qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit.",
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.",
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat."
]
sentences_rdd = sc.parallelize(sentences)

# Tokenize the sentences into words
words_rdd = sentences_rdd.flatMap(lambda line: line.split())

print(words_rdd.collect())

In [10]:
# Compute the frequency of each word
word_counts_rdd = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
print(word_counts_rdd.collect())

[('Spark', 2), ('is', 1), ('unified', 1), ('analytics', 1), ('engine', 1), ('It', 1), ('provides', 1), ('general-purpose', 1), ('cluster-computing', 1), ('framework', 1), ('are', 1), ('fundamental', 1), ('supports', 1), ('Python', 1), ('a', 2), ('fast', 1), ('and', 2), ('RDDs', 1), ('the', 1), ('data', 1), ('structure', 1), ('many', 1), ('languages', 1), ('including', 1), ('Scala', 1)]


In [11]:
# Find the most common word
most_common_word = word_counts_rdd.takeOrdered(1, key=lambda x: -x[1])
print("Most common word:", most_common_word)

Most common word: [('Spark', 2)]


In [12]:
# Compute the average word length
total_length_rdd = words_rdd.map(lambda word: len(word)).reduce(lambda a, b: a + b)
total_words = words_rdd.count()
average_word_length = total_length_rdd / total_words if total_words > 0 else 0
print("Average word length:", average_word_length)

Average word length: 6.142857142857143


In [ ]:
# Sort the RDD by word length in ascending order
sorted_words_by_length = words_rdd.sortBy(lambda word: len(word))

# Get the shortest word
shortest_word = sorted_words_by_length.first()

# Get the longest word
longest_word = sorted_words_by_length.sortBy(lambda word: len(word), ascending=False).first()

# Print the results
print("Shortest word:", shortest_word)
print("Longest word:", longest_word)

In [ ]:
# Alternative solution using reduce
# Find the shortest word
shortest_word = words_rdd.reduce(lambda a, b: a if len(a) < len(b) else b)
# Find the longest word
longest_word = words_rdd.reduce(lambda a, b: a if len(a) > len(b) else b)

# Print the results
print("Shortest word:", shortest_word)
print("Longest word:", longest_word)

In [13]:
# Stop the SparkContext
sc.stop()